In [1]:
# Install required libraries
!pip install openai gradio

# Import libraries
import openai
import gradio as gr

# 🔑 Enter your OpenAI API key here
openai.api_key = "YOUR_API_KEY"

# Function to generate roadmap
def generate_roadmap(goal, days, hours):
    prompt = f"""
    You are a Student Roadmap Generator.
    Create a clear day-by-day learning plan for the goal: {goal}.
    Total timeframe: {days} days.
    Student can study {hours} hours per day.
    Keep it motivational, structured, and easy to follow.
    Show tasks under each day.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.7
    )
    plan = response.choices[0].message["content"]
    return plan

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎓 Student Learning Path Generator")
    gr.Markdown("Generate your personalized study roadmap in seconds ✨")

    goal = gr.Textbox(label="🎯 Goal (e.g., UPSC in 30 days, Python in 7 days)")
    days = gr.Number(label="⏳ Timeframe (days)", value=30)
    hours = gr.Number(label="⌛ Study Hours per Day", value=3)
    output = gr.Textbox(label="📅 Your Study Roadmap", lines=20)

    btn = gr.Button("🚀 Generate Roadmap")
    btn.click(fn=generate_roadmap, inputs=[goal, days, hours], outputs=output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b10ff4e32d396f7861.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
# Install libraries
!pip install openai gradio

import openai
import gradio as gr
import datetime

# 🔑 Your OpenAI API Key
openai.api_key = "YOUR_API_KEY"

# Memory for progress tracking
user_progress = {"completed": 0, "total": 0, "streak": 0, "last_day": None}

# Function to generate roadmap
def generate_roadmap(goal, days, hours):
    prompt = f"""
    You are a Student Roadmap Generator.
    Create a structured day-by-day study plan for: {goal}.
    Duration: {days} days.
    Study time: {hours} hours/day.
    Keep it motivational and easy to follow.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.7
    )
    plan = response.choices[0].message["content"]

    # Split by days for task tracking
    tasks = []
    for line in plan.split("\n"):
        if line.strip():
            tasks.append(line.strip())
    user_progress["total"] = len(tasks)
    user_progress["completed"] = 0
    return "\n".join(tasks)

# Function to mark task completion
def mark_task(task):
    today = datetime.date.today()
    if user_progress["last_day"] != today:
        user_progress["streak"] += 1
        user_progress["last_day"] = today

    user_progress["completed"] += 1
    progress = (user_progress["completed"] / user_progress["total"]) * 100 if user_progress["total"] > 0 else 0
    return f"✅ Task '{task}' completed!\n\nProgress: {progress:.2f}%\n🔥 Streak: {user_progress['streak']} days"

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🎓 Student Learning Path Generator with Progress Tracker")
    gr.Markdown("Plan your journey, stay consistent, and track your streaks 🔥")

    with gr.Row():
        goal = gr.Textbox(label="🎯 Goal (e.g., UPSC in 30 days, Python in 7 days)")
        days = gr.Number(label="⏳ Days", value=30)
        hours = gr.Number(label="⌛ Hours/Day", value=3)

    roadmap_output = gr.Textbox(label="📅 Study Roadmap", lines=20)
    btn_generate = gr.Button("🚀 Generate Roadmap")

    task_input = gr.Textbox(label="✍️ Enter Task to Mark Complete")
    btn_task = gr.Button("✅ Mark Task as Done")
    task_status = gr.Textbox(label="📊 Progress & Streaks")

    btn_generate.click(fn=generate_roadmap, inputs=[goal, days, hours], outputs=roadmap_output)
    btn_task.click(fn=mark_task, inputs=[task_input], outputs=task_status)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://822665e530291fa6e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Colab-ready: Roadmap generator + interactive checkboxes + progress & streaks
# Paste into one Colab cell and run.

!pip install --quiet --upgrade openai gradio

import os, re, json, datetime, traceback
import openai
import gradio as gr

# ----------------------------
# CONFIG: provide your OpenAI key here OR set the COLAB environment variable:
# Option A (recommended): set in a separate Colab cell:
#   import os
#   os.environ["OPENAI_API_KEY"] = "sk-..."
# Option B: paste below (less secure)
openai.api_key = os.environ.get("OPENAI_API_KEY") or "YOUR_API_KEY"
# ----------------------------

def _safe_parse_json_array(text):
    """
    Try to find and parse the first JSON array in the model output.
    Return Python object or None.
    """
    try:
        m = re.search(r'(\[.*\])', text, re.S)
        if m:
            return json.loads(m.group(1))
    except Exception:
        pass
    return None

def generate_roadmap(goal, days, hours):
    """Try to call OpenAI to get a day-by-day plan. Return:
       - roadmap_text (string)
       - checkbox update for Gradio (choices)
       - tasks list (state)
       - progress state dict
    """
    try:
        days_i = max(1, int(days))
    except Exception:
        days_i = 7
    try:
        hours_f = float(hours)
    except Exception:
        hours_f = 1.0

    system = (
        "You are a helpful Student Roadmap Generator. "
        "Produce a JSON array where each item is {\"day\": <int>, \"tasks\": [\"task1\", \"task2\", ...]}. "
        "Keep tasks short (5-8 words each). Do not add any extra prose outside the JSON array."
    )
    user = f"Goal: {goal}\nDuration days: {days_i}\nHours per day: {hours_f}\nReturn only the JSON array as specified."

    try:
        # Try the API call
        resp = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": user},
            ],
            temperature=0.2,
            max_tokens=1200
        )
        content = resp["choices"][0]["message"]["content"].strip()
        parsed = _safe_parse_json_array(content)
        if parsed and isinstance(parsed, list) and len(parsed) > 0:
            tasks = []
            for item in parsed:
                day = item.get("day") if isinstance(item.get("day"), int) else len(tasks) + 1
                tlist = item.get("tasks") or []
                if isinstance(tlist, list):
                    tasks.append(f"Day {day}: " + " | ".join(tlist))
                else:
                    tasks.append(f"Day {day}: {str(tlist)}")
            roadmap_text = "\n".join(tasks)
        else:
            # If the model didn't return clean JSON, fallback to splitting
            lines = [ln.strip() for ln in content.splitlines() if ln.strip()]
            if len(lines) >= days_i:
                # Take first N lines
                tasks = [f"Day {i+1}: {lines[i]}" for i in range(min(days_i, len(lines)))]
            else:
                # Generic fallback generation
                tasks = [f"Day {i+1}: Study core topic {i+1}" for i in range(days_i)]
            roadmap_text = "\n".join(tasks)

    except Exception as e:
        # Print error for debugging in Colab output
        print("OpenAI API error (falling back to mocked plan):\n", traceback.format_exc())
        # Mock plan (safe demo mode)
        tasks = [f"Day {i+1}: Example tasks to make progress on '{goal}'" for i in range(days_i)]
        roadmap_text = "\n".join(tasks)

    # initialize progress state
    progress_state = {
        "completed": 0,
        "total": len(tasks),
        "streak": 0,
        "last_day": None  # iso date string
    }

    # Return: roadmap display text, checkbox update, tasks list, progress state
    return (roadmap_text,
            gr.CheckboxGroup.update(choices=tasks, value=[]),
            tasks,
            progress_state)

def update_progress(selected_tasks, tasks_state, progress_state):
    """selected_tasks: list of checked strings from CheckboxGroup
       tasks_state: list of all tasks (from state)
       progress_state: dict state
    """
    try:
        today = datetime.date.today().isoformat()
        completed = len(selected_tasks) if selected_tasks else 0
        total = len(tasks_state) if tasks_state else 0

        # streak logic: if user completes at least one new task today and last_day != today, increment streak
        # We detect a 'new activity today' when completed > progress_state['completed'] and last_day != today
        if total > 0:
            if progress_state.get("last_day") != today and completed > progress_state.get("completed", 0):
                progress_state["streak"] = progress_state.get("streak", 0) + 1
                progress_state["last_day"] = today

        progress_state["completed"] = completed
        progress_state["total"] = total

        pct = (completed / total * 100) if total > 0 else 0.0
        remaining = total - completed

        status = (
            f"✅ Progress: {pct:.2f}% ({completed}/{total})\n"
            f"🔥 Streak: {progress_state.get('streak', 0)} day(s)\n"
            f"⏳ Remaining tasks: {remaining}"
        )
    except Exception as e:
        status = "Error updating progress: " + str(e)

    return status, progress_state

def reset_all():
    return "", gr.CheckboxGroup.update(choices=[], value=[]), [], {"completed":0,"total":0,"streak":0,"last_day":None}, "Progress reset."

# -----------------------
# GRADIO UI
# -----------------------
with gr.Blocks(title="Student Learning Path Generator (Colab demo)") as demo:
    gr.Markdown("<h2>🎓 Student Learning Path Generator — Colab Demo</h2>")
    with gr.Row():
        goal = gr.Textbox(label="🎯 Goal (e.g., 'UPSC in 30 days', 'Python in 7 days')", placeholder="Type goal here...")
        days = gr.Number(label="⏳ Duration (days)", value=7)
        hours = gr.Number(label="⌛ Hours per day", value=2)

    with gr.Row():
        btn_gen = gr.Button("🚀 Generate Roadmap")
        btn_reset = gr.Button("🔄 Reset")

    roadmap_text = gr.Textbox(label="📅 Roadmap (generated)", lines=12)
    roadmap_check = gr.CheckboxGroup(choices=[], label="✔️ Tasks (check to mark complete)")
    tasks_state = gr.State([])   # list of tasks
    progress_state = gr.State({"completed":0,"total":0,"streak":0,"last_day":None})

    with gr.Row():
        btn_update = gr.Button("📈 Update Progress")
        status_out = gr.Textbox(label="📊 Progress & Streaks", lines=6)

    # wiring
    btn_gen.click(fn=generate_roadmap, inputs=[goal, days, hours],
                  outputs=[roadmap_text, roadmap_check, tasks_state, progress_state])
    btn_update.click(fn=update_progress, inputs=[roadmap_check, tasks_state, progress_state],
                     outputs=[status_out, progress_state])
    btn_reset.click(fn=reset_all, inputs=None, outputs=[roadmap_text, roadmap_check, tasks_state, progress_state, status_out])

demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 23.4 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ed77aa21161deace9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
